# Notebook to visualize computation of Landmarked Region

### Imports

In [ ]:
import sys
from copy import deepcopy

import cv2
import matplotlib.pyplot as plt
import numpy as np

sys.path.append("../")
from src.adnet import compute_landmarks
from src.ssd import ssd_detect
from src.utils import draw_bboxes_and_keypoints, crop_and_resize

%load_ext autoreload
%autoreload 2

### Face Detection

In [ ]:
img_p = "../data/009_03.jpg"
img_in = cv2.imread(img_p)
bboxes = ssd_detect(
    img_in,
    model_p="../checkpoints/ssd/ssd_facedetect.caffemodel",
    prototxt_p="../checkpoints/ssd/ssd_facedetect.prototxt.txt",
)
n_detections = len(bboxes)

if n_detections >= 1:
    # Use biggest bbox
    max_area = 0
    max_idx = 0
    for idx in range(len(bboxes)):
        bbox = bboxes[idx]
        width = bbox[2]
        height = bbox[3]
        area = width * height
        if area > max_area:
            max_area = area
            max_idx = idx
else:
    print("Did not detect any faces")

# Show image with detections
img_out = draw_bboxes_and_keypoints(img_in, bboxes, keypoints_all=None)
plt.imshow(cv2.cvtColor(img_out, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Crop image to bounding box
bbox = bboxes[max_idx]
cropped_img = crop_and_resize(img_in, bbox, (616, 616))
plt.imshow(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
plt.show()

### Landmark Detection

In [ ]:
# Helper function
def draw_points(img: np.ndarray, points: np.ndarray, name: str = "default"):
    for idx in range(len(points)):
        x, y = points[idx]
        cv2.circle(img, (x, y), 3, (255, 0, 0), cv2.FILLED)
        cv2.putText(img, name, (x + 2, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    return img


# Compute and plot landmarks
img = cropped_img
model_p = "../checkpoints/adnet/ADNet.onnx"
landmarks = compute_landmarks(img, model_p)
# print(landmarks.shape)
width, height, channels = img.shape
img_landmarks = deepcopy(img)
for idx in range(len(landmarks)):
    x, y = landmarks[idx]
    cv2.circle(img_landmarks, (x, y), 3, (255, 0, 0), cv2.FILLED)
    cv2.putText(img_landmarks, str(idx), (x + 2, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
plt.imshow(cv2.cvtColor(img_landmarks, cv2.COLOR_BGR2RGB))
# plt.savefig("../output/01_landmarks.png", bbox_inches="tight")
plt.show()

### Break down GetFaceMask() function into individual parts

In [ ]:
# Compute convex hull from selection of landmarks
hull_points = cv2.convexHull(landmarks).squeeze()

# Drawing function
img_hullpoints = draw_points(deepcopy(img), hull_points, "hl")
plt.imshow(cv2.cvtColor(img_hullpoints, cv2.COLOR_BGR2RGB))
# plt.savefig("../output/02_convex_hull.png", bbox_inches="tight")
plt.show()

In [ ]:
# Generate mask from convex hull
mask = np.zeros((height, width), dtype=np.uint8)
cv2.fillConvexPoly(mask, np.array(hull_points, dtype=np.int32), 1)

# Show mask
plt.imshow(mask, cmap="gray")
plt.show()

### Addition: Blend mask on input image

In [ ]:
# Plot face_mask on img
face_mask = mask
face_mask = cv2.cvtColor(face_mask, cv2.COLOR_GRAY2BGR)  # Extend face_mask channels
face_mask[:, :, 2] = face_mask[:, :, 2] * 120.0
img_rgb = cv2.cvtColor(deepcopy(img), cv2.COLOR_BGR2RGB)
img_mask = cv2.addWeighted(img_rgb, 0.7, face_mask, 0.3, 0.0)

plt.imshow(img_mask)
# plt.savefig("../output/03_mask_on_img.png", bbox_inches="tight")
plt.show()

In [ ]:
# Create subplot with all steps
from typing import List

fig, axs = plt.subplots(1, 3, figsize=(12, 6))
axs: List[plt.Axes] = np.ravel(axs)

for ax in axs:
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

axs[0].imshow(cv2.cvtColor(img_landmarks, cv2.COLOR_BGR2RGB))
axs[0].set_title("ADNet Landmarks")
axs[1].imshow(cv2.cvtColor(img_hullpoints, cv2.COLOR_BGR2RGB))
axs[1].set_title("OpenCV convexHull")
axs[2].imshow(img_mask)
axs[2].set_title("Landmarked Region")

plt.tight_layout()
# plt.savefig("../output/ofiq_convexHull.png", bbox_inches="tight")
plt.show()